In [1]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import torch
print(os.getcwd())

from torchrbpnet.data import tfrecord_to_dataloader, dummy_dataloader
from torchrbpnet.data import datasets

# dataloader = torch.utils.data.DataLoader(datasets.TFIterableDataset('../example/head.tfrecord', batch_size=2, shuffle=100), batch_size=None)

/home/marc/PhD/TorchRBPNet/notebooks


In [3]:
esm_masked_dataset = datasets.MeanESMEmbeddingMaskedTFIterableDataset(embedding_matrix_filepath='../example/esm2_t33_650M_UR50D.ENCODE.idx2mean.pt', filepath='../example/head.tfrecord', batch_size=2, shuffle=100)
dataloader = torch.utils.data.DataLoader(esm_masked_dataset, batch_size=None)
print(next(iter(dataloader))[0].keys())

dict_keys(['sequence', 'embedding'])


/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/torchrbpnet/data/datasets.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tf.nest.map_structure(lambda x: torch.tensor(x).to(torch.float32), example)


In [4]:
# %%
import torch
import torch.nn as nn
import pytorch_lightning as pl

import torchmetrics
from torchrbpnet.losses import MultinomialNLLLossFromLogits
from torchrbpnet.metrics import BatchedPCC, MultinomialNLLFromLogits
from torchrbpnet.networks import MultiRBPNet, ProteinEmbeddingMultiRBPNet

class Model(pl.LightningModule):
    def __init__(self, network, _example_input_shape=None):
        super().__init__()
        self.network = network
        self.loss_fn = MultinomialNLLLossFromLogits()
        # self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()}) # This has to be wrapped in a nn.ModuleDict (otherwise .to_device has to be called manually on metrics)
        self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()})
        # self.example_input_array = torch.rand(*_example_input_shape)
    
    def forward(self, *args, **kwargs):
        return self.network(*args, **kwargs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx, **kwargs):
        inputs, y = batch
        y = y['total']
        y_pred = self.forward(inputs)
        # print(y_pred)
        loss = self.loss_fn(y, y_pred, dim=-2)
        self.compute_and_log_metics(y_pred, y, partition='train')
        return loss
    
    def training_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_step(self, batch, batch_idx):
        inputs, y = batch
        y = y['total']
        y_pred = self.forward(inputs)
        self.compute_and_log_metics(y_pred, y, partition='val')
    
    def compute_and_log_metics(self, y_pred, y, partition=None):
        on_step = False
        if partition == 'train':
            on_step = True

        for name, metric in self.metrics.items():
            metric(y_pred, y)
            self.log(f'{partition}/{name}', metric.compute(), on_step=on_step, on_epoch=True, prog_bar=False)
    
    def _reset_metrics(self):
        for metric in self.metrics.values():
            metric.reset()

# model = Model(network=MultiRBPNet(n_tasks=223)) # _example_input_shape=next(iter(dataloader))[0].shape
model = Model(network=ProteinEmbeddingMultiRBPNet())

In [5]:
example = next(iter(dataloader))

In [6]:
model(example[0]).shape

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,


torch.Size([2, 1000, 223])

In [7]:
import datetime
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, RichProgressBar, LearningRateMonitor

root_log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
loggers = [
    pl_loggers.TensorBoardLogger(root_log_dir+'/tensorboard', name='', version='', log_graph=True),
    # pl_loggers.CSVLogger(root_log_dir+'/tensorboard', name='', version=''),
]

checkpoint_callback = ModelCheckpoint(dirpath=f'{root_log_dir}/checkpoints', every_n_epochs=1, save_last=True)

early_stop_callback = EarlyStopping(monitor="val/loss", min_delta=0.00, patience=3, verbose=False, mode="min")

bar = RichProgressBar()

trainer = pl.Trainer(default_root_dir=root_log_dir, max_epochs=2, logger=loggers, callbacks=[checkpoint_callback, early_stop_callback, LearningRateMonitor('step', log_momentum=True)])
trainer.fit(model=model, train_dataloaders=dataloader, val_dataloaders=dataloader)
torch.save(model.network, 'test.pt')

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name    | Type                         | Params
---------------------------------------------------------
0 | network | ProteinEmbeddingMultiRBPNet  | 2.2 M 
1 | loss_fn | MultinomialNLLLossFromLogits | 0     
2 | metrics | ModuleDict                   | 0     
---------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.704     Total estimated model params size (MB)
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/loggers/tensorboard.py:188: UserWarning: Could not log computati

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 10.82it/s]

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:84: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: : 2it [00:00,  2.05it/s, loss=4.33, v_num=]

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:84: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 1: : 6it [00:02,  2.56it/s, loss=4.08, v_num=]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: : 6it [00:02,  2.36it/s, loss=4.08, v_num=]
